<div style="background-color: #faf3e0; padding: 20px; border-radius: 12px; border: 2px solid #6a0dad;">
    <h2 style="color: #6a0dad; text-align: center; font-family: 'Georgia', serif; font-size: 24px; margin-bottom: 10px;">
        INTRODUCTION: THE PERPLEXITY PERMUTATION PUZZLE
    </h2>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        Welcome to the <strong>Perplexity Permutation Puzzle</strong>, a unique challenge that combines the art of language with the science of optimization. In this competition, we are tasked with rearranging scrambled holiday-themed text sequences to minimize perplexity—a measure of how well a language model predicts the sequence. The goal is to transform chaotic word arrangements into coherent, meaningful passages that delight readers and showcase the power of natural language processing (NLP) and optimization techniques.
    </p>
    <h3 style="color: #6a0dad; font-family: 'Georgia', serif; font-size: 20px; margin-top: 15px;">
        Key Highlights of This Notebook
    </h3>
    <ul style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        <li><strong>Advanced Optimization Techniques:</strong> This notebook employs a multi-phase optimization approach, including <strong>local window optimization</strong>, <strong>global jumps</strong>, <strong>chunk swaps</strong>, and <strong>global shuffling</strong>, to iteratively improve text sequences.</li>
        <li><strong>State-of-the-Art Language Models:</strong> Leveraging pre-trained models like <strong>Gemma 2 9B</strong>, we ensure accurate and efficient perplexity calculations, enabling precise evaluation of text coherence.</li>
        <li><strong>Dynamic Parameter Tuning:</strong> The optimization process dynamically adjusts parameters such as window size and early stopping thresholds, ensuring adaptability to sequences of varying lengths and complexities.</li>
        <li><strong>Comprehensive Evaluation:</strong> Detailed metrics, including success rates, improvement percentages, and time taken for each phase, provide valuable insights into the optimization process.</li>
    </ul>
    <h3 style="color: #6a0dad; font-family: 'Georgia', serif; font-size: 20px; margin-top: 15px;">
        Why This Approach?
    </h3>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        The Perplexity Permutation Puzzle is more than just a competition—it's an opportunity to explore the intersection of language, optimization, and machine learning. By combining advanced NLP techniques with robust optimization strategies, this notebook aims to deliver solutions that are not only competitive but also demonstrate the transformative potential of modern AI. Our approach ensures that every sequence is optimized for clarity, coherence, and reader satisfaction.
    </p>
    <h3 style="color: #6a0dad; font-family: 'Georgia', serif; font-size: 20px; margin-top: 15px;">
        How to Use This Notebook
    </h3>
    <ol style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        <li><strong>Setup and Installation:</strong> Begin by installing the necessary libraries and configuring the environment to ensure seamless execution.</li>
        <li><strong>Data Loading:</strong> Load the scrambled text sequences and prepare them for optimization.</li>
        <li><strong>Optimization Process:</strong> Follow the step-by-step optimization pipeline to rearrange and improve each sequence.</li>
        <li><strong>Evaluation:</strong> Analyze the perplexity scores and track improvements across optimization phases.</li>
        <li><strong>Submission:</strong> Generate the final submission file containing the optimized sequences, ready for evaluation.</li>
    </ol>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif; text-align: center; margin-top: 20px;">
        Let’s embark on this journey to solve the <strong>Perplexity Permutation Puzzle</strong> and create text sequences that inspire and delight! 🎅
    </p>
</div>

In [1]:
pip install --upgrade ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install numpy 
!pip install pandas 
!pip install torch 
!pip install transformers 
!pip install nltk
!pip install tqdm 

<div style="background-color: #faf3e0; padding: 20px; border-radius: 12px; border: 2px solid #6a0dad;">
    <h2 style="color: #6a0dad; text-align: center; font-family: 'Georgia', serif; font-size: 24px; margin-bottom: 10px;">
        I. IMPORTS AND ENVIRONMENT SETUP
    </h2>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        This section includes all the necessary imports for the project, such as libraries for data manipulation (<code>numpy</code>, <code>pandas</code>), deep learning (<code>torch</code>, <code>transformers</code>), and text processing (<code>nltk</code>). Additionally, it sets up the environment for optimal performance, including downloading NLTK stopwords and configuring CUDA for GPU usage.
    </p>
</div>

In [3]:
import numpy as np
import pandas as pd
import torch
import transformers
import itertools
from tqdm import tqdm
import os
from math import exp
from collections import defaultdict
from typing import List, Tuple, Dict, Set
import nltk
from nltk.corpus import stopwords
from datetime import datetime
import random
import time

# Download stopwords if not already present
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Environment setup
os.environ['OMP_NUM_THREADS'] = '2'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
DEVICE = torch.device('cuda')

<div style="background-color: #faf3e0; padding: 20px; border-radius: 12px; border: 2px solid #6a0dad;">
    <h2 style="color: #6a0dad; text-align: center; font-family: 'Georgia', serif; font-size: 24px; margin-bottom: 10px;">
        II. PERPLEXITY CALCULATOR
    </h2>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        The <code>PerplexityCalculator</code> class is a key component of the project, designed to calculate the perplexity of a given text using a pre-trained language model. Perplexity is a metric that evaluates how well a language model predicts a sequence of words, with lower values indicating better performance. This class is essential for optimizing the scrambled text sequences in the competition.
    </p>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        Key features of the <code>PerplexityCalculator</code> class:
    </p>
    <ul style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        <li><strong>Initialization:</strong> Loads a pre-trained tokenizer and language model, and sets up the loss function (<code>CrossEntropyLoss</code>).</li>
        <li><strong>Perplexity Calculation:</strong> Computes the perplexity of a given text by evaluating the model's predictions and calculating the loss.</li>
        <li><strong>Caching Mechanism:</strong> Stores previously computed perplexity values to avoid redundant calculations for the same text.</li>
        <li><strong>Efficiency:</strong> Uses GPU acceleration (via <code>torch</code>) for faster computation, making it suitable for large-scale text processing.</li>
    </ul>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        This class plays a critical role in evaluating and optimizing the scrambled text sequences, ensuring that the final output has the lowest possible perplexity.
    </p>
</div>

In [4]:
class PerplexityCalculator:
    def __init__(self, model_path: str):
        print("\n🔄 Initializing PerplexityCalculator...")
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
        self.model = transformers.AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            torch_dtype=torch.float16
        )
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        self.model.eval()
        self.cache = {}
        print("✅ Model loaded successfully")
        
    def get_perplexity(self, text: str) -> float:
        if text in self.cache:
            return self.cache[text]
            
        with torch.no_grad():
            text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
            model_inputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False)
            model_inputs = {k: v.to(self.model.device) for k, v in model_inputs.items()}
            
            logits = self.model(**model_inputs, use_cache=False)['logits']
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = model_inputs['input_ids'][..., 1:].contiguous()
            
            loss = self.loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1)
            )
            perplexity = exp(loss.sum().item() / len(loss))
            self.cache[text] = perplexity
            return perplexity

<div style="background-color: #faf3e0; padding: 20px; border-radius: 12px; border: 2px solid #6a0dad;">
    <h2 style="color: #6a0dad; text-align: center; font-family: 'Georgia', serif; font-size: 24px; margin-bottom: 10px;">
        III. ADVANCED SEQUENCE SOLVER
    </h2>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        The <code>AdvancedSequenceSolver</code> class is the core of the optimization process. It uses a combination of strategies to rearrange scrambled text sequences, aiming to minimize perplexity. This class integrates multiple optimization techniques, including local window optimization, global jumps, chunk swaps, and global shuffling, to iteratively improve the text sequence.
    </p>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        Key features of the <code>AdvancedSequenceSolver</code> class:
    </p>
    <ul style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        <li>
            <strong>Local Window Optimization:</strong> Rearranges words within small windows to find better sequences, with early stopping to avoid unnecessary computations.
            <br>
            <strong>Example:</strong> For the sequence <code>"reindeer sleep walk the night"</code>, the solver might rearrange the window <code>"sleep walk the"</code> to <code>"walk sleep the"</code> if it reduces perplexity.
        </li>
        <li>
            <strong>Global Jumps:</strong> Moves groups of words to different positions in the sequence, exploring larger-scale rearrangements.
            <br>
            <strong>Example:</strong> In the sequence <code>"reindeer sleep walk the night"</code>, the solver might move the group <code>"sleep walk"</code> to the end, resulting in <code>"reindeer the night sleep walk"</code>.
        </li>
        <li>
            <strong>Chunk Swaps:</strong> Swaps chunks of words to test different configurations, improving the overall flow of the text.
            <br>
            <strong>Example:</strong> For the sequence <code>"reindeer sleep walk the night"</code>, the solver might swap the chunks <code>"reindeer sleep"</code> and <code>"the night"</code>, resulting in <code>"the night walk reindeer sleep"</code>.
        </li>
        <li>
            <strong>Global Shuffling:</strong> Randomly shuffles the entire sequence to explore entirely new configurations, with early stopping if no improvements are found.
            <br>
            <strong>Example:</strong> The sequence <code>"reindeer sleep walk the night"</code> might be shuffled to <code>"night the walk sleep reindeer"</code> to explore a completely new arrangement.
        </li>
        <li>
            <strong>Logging and Tracking:</strong> Tracks improvements, success rates, and time taken for each phase, providing detailed insights into the optimization process.
            <br>
        </li>
    </ul>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        The class also includes a <code>_log_phase_start</code> and <code>_log_phase_end</code> method to log the progress of each optimization phase, as well as a <code>_update_best</code> method to update the best score and text whenever an improvement is found.
    </p>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        This class is designed to efficiently explore the space of possible word arrangements, ensuring that the final sequence has the lowest possible perplexity.
    </p>
</div>

In [5]:
class AdvancedSequenceSolver:
    def __init__(self, scorer: PerplexityCalculator):
        self.scorer = scorer
        self.word_groups = defaultdict(set)
        self.improvements_count = 0
        self.start_time = None
        self.best_score = float('inf')
        self.best_text = None
        self.phase_improvements = defaultdict(int)
        self.phase_times = {}
        self.attempts_per_phase = defaultdict(int)
        
    def _log_phase_start(self, phase_name: str) -> float:
        """Log the start of an optimization phase"""
        phase_start = time.time()
        total_time = phase_start - self.start_time
        print(f"\n{'='*80}")
        print(f"⏳ Phase: {phase_name}")
        print(f"   Time elapsed: {total_time:.2f}s")
        print(f"   Current best score: {self.best_score:.2f}")
        return phase_start
        
    def _log_phase_end(self, phase_name: str, phase_start: float):
        """Log the end of an optimization phase"""
        phase_duration = time.time() - phase_start
        self.phase_times[phase_name] = phase_duration
        improvements = self.phase_improvements[phase_name]
        attempts = self.attempts_per_phase[phase_name]
        print(f"\n📊 Phase Complete: {phase_name}")
        print(f"   Duration: {phase_duration:.2f}s")
        print(f"   Attempts: {attempts}")
        print(f"   Improvements found: {improvements}")
        if improvements > 0:
            print(f"   Success rate: {(improvements/attempts)*100:.2f}%")
            print(f"   Average improvement per success: {(self.initial_score - self.best_score) / improvements:.2f}")

    def _update_best(self, text: str, score: float, strategy: str) -> bool:
        """Update best score if improvement found"""
        phase = strategy.split('-')[0]
        self.attempts_per_phase[phase] += 1
        
        if score < self.best_score:
            self.improvements_count += 1
            self.phase_improvements[phase] += 1
            improvement = self.best_score - score
            percent_improvement = (improvement / self.best_score) * 100
            
            print(f"\n🌟 Improvement #{self.improvements_count} (via {strategy}):")
            print(f"   Previous score: {self.best_score:.2f}")
            print(f"   New score: {score:.2f}")
            print(f"   Absolute improvement: {improvement:.2f}")
            print(f"   Relative improvement: {percent_improvement:.2f}%")
            print(f"   Current text: {text}")
            
            self.best_score = score
            self.best_text = text
            return True
        return False

    def window_optimization(self, words: List[str], max_window: int = 4, max_permutations_window: int = 100, max_attempts_without_improvement_window: int = 50) -> List[str]:
        print("\n🔍 Running local window optimization...")
        print(f"   Max window size: {max_window}")
        print(f"   Sequence length: {len(words)} words")
        best_local = words.copy()
        window_stats = defaultdict(int)
    
        # Randomize the starting position of the window
        start_positions = list(range(len(words) - max_window + 1))
        random.shuffle(start_positions)
    
        for window_size in range(2, max_window + 1):
            window_start = time.time()
            improvements_at_size = 0
            attempts = 0
            attempts_without_improvement = 0
    
            print(f"\n   Processing window size {window_size}:")
            for i in start_positions:  # Use randomized starting positions
                if i + window_size > len(words):
                    continue
                window = words[i:i + window_size]
                # Limit the number of permutations
                permutations = list(itertools.permutations(window))
                if len(permutations) > max_permutations_window:
                    permutations = random.sample(permutations, max_permutations_window)
    
                for perm in permutations:
                    attempts += 1
                    new_arrangement = best_local.copy()
                    new_arrangement[i:i + window_size] = perm
                    new_text = ' '.join(new_arrangement)
                    score = self.scorer.get_perplexity(new_text)
                    if self._update_best(new_text, score, f"window-{window_size}"):
                        improvements_at_size += 1
                        best_local = new_arrangement
                        attempts_without_improvement = 0
                    else:
                        attempts_without_improvement += 1
    
                    # Early stopping
                    if attempts_without_improvement >= max_attempts_without_improvement_window:
                        print(f"   Early stopping at window size {window_size}: No improvement in {max_attempts_without_improvement_window} attempts")
                        break
    
                if attempts_without_improvement >= max_attempts_without_improvement_window:
                    break
    
            window_time = time.time() - window_start
            print(f"   Window size {window_size} complete:")
            print(f"      Time taken: {window_time:.2f}s")
            print(f"      Attempts: {attempts}")
            print(f"      Improvements: {improvements_at_size}")
            if attempts > 0:
                print(f"      Success rate: {(improvements_at_size/attempts)*100:.2f}%")
    
        return best_local

    def try_global_jumps(self, words: List[str], max_group_size_jumps: int = 9, max_attempts_without_improvement_jumps: int = 50) -> List[str]:
        print("\n🦘 Running global jumps optimization...")
        print(f"   Sequence length: {len(words)} words")
        jump_stats = defaultdict(lambda: {'attempts': 0, 'improvements': 0})
        attempts_without_improvement = 0
    
        for group_size in range(1, max_group_size_jumps + 1):
            group_start = time.time()
            print(f"\n   Testing group size: {group_size}")
    
            # Randomize the order of starting positions
            start_positions = list(range(len(words) - group_size))
            random.shuffle(start_positions)
    
            for start in start_positions:
                word_group = words[start:start + group_size]
                # Randomize the order of target positions
                target_positions = list(range(len(words) - group_size))
                random.shuffle(target_positions)
    
                for new_pos in target_positions:
                    if new_pos != start:
                        jump_stats[group_size]['attempts'] += 1
                        new_arrangement = words.copy()
                        del new_arrangement[start:start + group_size]
                        new_arrangement[new_pos:new_pos] = word_group
                        new_text = ' '.join(new_arrangement)
                        score = self.scorer.get_perplexity(new_text)
                        if self._update_best(new_text, score, f"jump-{group_size}"):
                            jump_stats[group_size]['improvements'] += 1
                            words = new_arrangement
                            attempts_without_improvement = 0  # Reset counter
                        else:
                            attempts_without_improvement += 1
    
                        # Early stopping
                        if attempts_without_improvement >= max_attempts_without_improvement_jumps:
                            print(f"   Early stopping at group size {group_size}: No improvement in {max_attempts_without_improvement_jumps} attempts")
                            break
    
                if attempts_without_improvement >= max_attempts_without_improvement_jumps:
                    break
    
            group_time = time.time() - group_start
            attempts = jump_stats[group_size]['attempts']
            improvements = jump_stats[group_size]['improvements']
            print(f"   Group size {group_size} complete:")
            print(f"      Time taken: {group_time:.2f}s")
            print(f"      Attempts: {attempts}")
            print(f"      Improvements: {improvements}")
            if attempts > 0:
                print(f"      Success rate: {(improvements/attempts)*100:.2f}%")
    
        return words
    def try_chunk_swaps(self, words: List[str], chunk_sizes_swaps: List[int] = [2, 3, 4, 5, 6, 7, 8, 9], max_attempts_without_improvement_swaps: int = 50) -> List[str]:
        print("\n🔄 Running chunk swaps optimization...")
        print(f"   Sequence length: {len(words)} words")
        swap_stats = defaultdict(lambda: {'attempts': 0, 'improvements': 0})
        attempts_without_improvement = 0
    
        for size in chunk_sizes_swaps:
            size_start = time.time()
            print(f"\n   Testing chunk size: {size}")
    
            # Randomize the order of chunk positions
            chunk_positions = list(range(0, len(words) - size, size))
            random.shuffle(chunk_positions)
    
            for i in chunk_positions:
                for j in range(i + size, len(words) - size + 1, size):
                    swap_stats[size]['attempts'] += 1
                    new_arrangement = words.copy()
                    chunk1 = new_arrangement[i:i + size]
                    chunk2 = new_arrangement[j:j + size]
                    new_arrangement[i:i + size] = chunk2
                    new_arrangement[j:j + size] = chunk1
                    new_text = ' '.join(new_arrangement)
                    score = self.scorer.get_perplexity(new_text)
                    if self._update_best(new_text, score, f"swap-{size}"):
                        swap_stats[size]['improvements'] += 1
                        words = new_arrangement
                        attempts_without_improvement = 0  # Reset counter
                    else:
                        attempts_without_improvement += 1
    
                    # Early stopping
                    if attempts_without_improvement >= max_attempts_without_improvement_swaps:
                        print(f"   Early stopping at chunk size {size}: No improvement in {max_attempts_without_improvement_swaps} attempts")
                        break
    
                if attempts_without_improvement >= max_attempts_without_improvement_swaps:
                    break
    
            size_time = time.time() - size_start
            attempts = swap_stats[size]['attempts']
            improvements = swap_stats[size]['improvements']
            print(f"   Chunk size {size} complete:")
            print(f"      Time taken: {size_time:.2f}s")
            print(f"      Attempts: {attempts}")
            print(f"      Improvements: {improvements}")
            if attempts > 0:
                print(f"      Success rate: {(improvements/attempts)*100:.2f}%")
    
        return words

    def global_shuffle_phase(self, words: List[str], n_attempts_shuffle: int = 100, max_attempts_without_improvement_shuffle: int = 20) -> List[str]:
        """Global shuffle optimization with early stopping"""
        print(f"\n🎲 Running global shuffle optimization...")
        print(f"   Planned attempts: {n_attempts_shuffle}")
        print(f"   Sequence length: {len(words)} words")
        improvements = 0
        phase_start = time.time()
        attempts_without_improvement = 0
        
        for i in range(n_attempts_shuffle):
            if i % 10 == 0:
                elapsed = time.time() - phase_start
                if i > 0:
                    avg_time_per_attempt = elapsed / i
                    estimated_remaining = avg_time_per_attempt * (n_attempts_shuffle - i)
                    print(f"   Progress: {i}/{n_attempts_shuffle} attempts")
                    print(f"   Time elapsed: {elapsed:.2f}s")
                    print(f"   Estimated remaining time: {estimated_remaining:.2f}s")
                    print(f"   Improvements so far: {improvements}")
            
            shuffled = words.copy()
            random.shuffle(shuffled)
            new_text = ' '.join(shuffled)
            score = self.scorer.get_perplexity(new_text)
            if self._update_best(new_text, score, "shuffle"):
                improvements += 1
                words = shuffled
                attempts_without_improvement = 0  # Reset counter
            else:
                attempts_without_improvement += 1
            
            # Early stopping
            if attempts_without_improvement >= max_attempts_without_improvement_shuffle:
                print(f"   Early stopping: No improvement in {max_attempts_without_improvement_shuffle} attempts")
                break
        
        total_time = time.time() - phase_start
        print(f"\n   Shuffle phase complete:")
        print(f"   Total time: {total_time:.2f}s")
        print(f"   Attempts: {i + 1}")
        print(f"   Improvements: {improvements}")
        print(f"   Success rate: {(improvements/(i + 1))*100:.2f}%")
        print(f"   Average time per attempt: {total_time/(i + 1):.3f}s")
                    
        return words

    def optimize_sequence(
        self, 
        text: str, 
        # Hyperparameters for window optimization
        max_window: int = 4, 
        max_permutations_window: int = 500, 
        max_attempts_without_improvement_window: int = 50, 
        # Hyperparameters for global jumps
        max_group_size_jumps: int = 9, 
        max_attempts_without_improvement_jumps: int = 50,
        # Hyperparameters for chunk swaps
        chunk_sizes_swaps: List[int] = [2, 3, 4, 5, 6, 7, 8, 9], 
        max_attempts_without_improvement_swaps: int = 50,
        # Hyperparameters for global shuffling
        n_attempts_shuffle: int = 100, 
        max_attempts_without_improvement_shuffle: int = 20,
    ) -> str:
        """Main optimization function combining all strategies with enhanced parameters"""
        self.start_time = time.time()
        print("\n🚀 Starting advanced sequence optimization...")
        print(f"Initial text: {text}")
        
        words = text.split()
        self.best_text = text
        self.best_score = self.scorer.get_perplexity(text)
        self.initial_score = self.best_score
        print(f"Initial perplexity score: {self.best_score:.2f}")
        
        # Phase 1: Local window optimization
        phase_start = self._log_phase_start("Local Window Optimization")
        words = self.window_optimization(
            words, 
            max_window=max_window, 
            max_permutations_window=max_permutations_window, 
            max_attempts_without_improvement_window=max_attempts_without_improvement_window
        )
        self._log_phase_end("window", phase_start)
        
        # Phase 2: Global jumps
        phase_start = self._log_phase_start("Global Jumps")
        words = self.try_global_jumps(
            words,
            max_group_size_jumps = max_group_size_jumps,
            max_attempts_without_improvement_jumps=max_attempts_without_improvement_jumps
        )
        self._log_phase_end("jump", phase_start)
        
        # Phase 3: Chunk swaps
        phase_start = self._log_phase_start("Chunk Swaps")
        words = self.try_chunk_swaps(
            words, 
            chunk_sizes_swaps = chunk_sizes_swaps,
            max_attempts_without_improvement_swaps=max_attempts_without_improvement_swaps
        )
        self._log_phase_end("swap", phase_start)
        
        # Phase 4: Global shuffling
        phase_start = self._log_phase_start("Global Shuffling")
        words = self.global_shuffle_phase(
            words, 
            n_attempts_shuffle=50, 
            max_attempts_without_improvement_shuffle=max_attempts_without_improvement_shuffle, 
        )
        self._log_phase_end("shuffle", phase_start)
        
        # Phase 5: Final local optimization
        phase_start = self._log_phase_start("Final Local Optimization")
        words = self.window_optimization(
            words, 
            max_window=max_window, 
            max_permutations_window=max_permutations_window, 
            max_attempts_without_improvement_window=max_attempts_without_improvement_window
        )
        self._log_phase_end("window", phase_start)
        
        # Final summary
        total_duration = time.time() - self.start_time
        total_improvement = self.initial_score - self.best_score
        print("\n📈 Optimization Summary:")
        print(f"Total time: {total_duration:.2f} seconds")
        print(f"Total improvements: {self.improvements_count}")
        print(f"Initial score: {self.initial_score:.2f}")
        print(f"Final score: {self.best_score:.2f}")
        print(f"Total improvement: {total_improvement:.2f} ({(total_improvement/self.initial_score)*100:.2f}%)")
        print("\nPhase Statistics:")
        for phase, improvements in self.phase_improvements.items():
            time_taken = self.phase_times.get(phase, 0)
            attempts = self.attempts_per_phase.get(phase, 0)
            if attempts > 0:
                success_rate = (improvements/attempts)*100
                print(f"- {phase}:")
                print(f"  • {improvements} improvements in {time_taken:.2f}s")
                print(f"  • Success rate: {success_rate:.2f}% ({improvements}/{attempts})")
            else:
                print(f"- {phase}: No attempts made")
        
        return self.best_text

<div style="background-color: #faf3e0; padding: 20px; border-radius: 12px; border: 2px solid #6a0dad;">
    <h2 style="color: #6a0dad; text-align: center; font-family: 'Georgia', serif; font-size: 24px; margin-bottom: 10px;">
        IV. OPTIMIZE SUBMISSION FUNCTION
    </h2>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        The <code>optimize_submission</code> function is the main driver for optimizing the scrambled text sequences in the dataset. It takes a DataFrame containing the scrambled sequences and a path to the pre-trained language model as input. For each sequence, it applies the <code>AdvancedSequenceSolver</code> to rearrange the words and minimize perplexity.
    </p>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        Key steps performed by the <code>optimize_submission</code> function:
    </p>
    <ul style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        <li><strong>Initialization:</strong> Creates an instance of <code>PerplexityCalculator</code> and <code>AdvancedSequenceSolver</code> to handle perplexity calculations and sequence optimization.</li>
        <li><strong>Sequence Processing:</strong> Iterates over each sequence in the DataFrame, calculates its length, and dynamically adjusts the maximum window size for optimization based on the sequence length.</li>
        <li><strong>Optimization:</strong> Applies the <code>optimize_sequence</code> method from <code>AdvancedSequenceSolver</code> to rearrange the words in the sequence, aiming to minimize perplexity.</li>
        <li><strong>Progress Tracking:</strong> Uses <code>tqdm</code> to display a progress bar and logs detailed information about each sequence being processed.</li>
        <li><strong>Result Storage:</strong> Updates the DataFrame with the optimized text sequences, which are then returned as the final output.</li>
    </ul>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        This function is designed to efficiently process multiple sequences, ensuring that each one is optimized for the lowest possible perplexity while providing clear progress updates.
    </p>
</div>

<div style="background-color: #faf3e0; padding: 20px; border-radius: 12px; border: 2px solid #6a0dad;">
    <h2 style="color: #6a0dad; text-align: center; font-family: 'Georgia', serif; font-size: 24px; margin-bottom: 10px;">
        V. SETUP, DATA LOADING, AND OPTIMIZATION EXECUTION
    </h2>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        This section sets up the environment, loads the dataset, and executes the optimization process. It defines the paths for the pre-trained model and output file, loads the scrambled text sequences, and applies the optimization pipeline to generate the final submission.
    </p>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        Key steps in this section:
    </p>
    <ul style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        <li><strong>Path Setup:</strong> Defines the paths for the pre-trained model (<code>MODEL_PATH</code>) and the output submission file (<code>OUTPUT_PATH</code>).</li>
        <li><strong>Data Loading:</strong> Loads the scrambled text sequences into a DataFrame, adds an <code>id</code> column, and reorders the columns for consistency.</li>
        <li><strong>Optimization Execution:</strong> Calls the <code>optimize_submission</code> function to optimize each sequence in the dataset and saves the results to a CSV file.</li>
        <li><strong>Error Handling:</strong> Includes a <code>try-except</code> block to catch and report any errors that occur during the optimization process.</li>
    </ul>
    <p style="color: #333333; font-size: 16px; line-height: 1.6; font-family: 'Arial', sans-serif;">
        This section ties everything together, ensuring that the optimization pipeline runs smoothly and produces the final submission file for the competition.
    </p>
</div>

In [6]:
def optimize_submission(df: pd.DataFrame, model_path: str) -> pd.DataFrame:
    scorer = PerplexityCalculator(model_path)
    solver = AdvancedSequenceSolver(scorer)

    # Create a new DataFrame to store id, optimized_text, and perplexity
    results_df = pd.DataFrame(columns=["id", "optimized_text", "perplexity"])
    
    print(f"\n🎯 Starting optimization for {len(df)} sequences...")
    for idx in tqdm(range(len(df))):
        print(f"\n{'='*80}")
        print(f"📝 Processing sequence {idx+1}/{len(df)}")
        
        original_text = df.loc[idx, 'text']
        words_count = len(original_text.split())
        max_window = 7
        print(f"Sequence length: {words_count} words -> Using max window size: {max_window}")
        
        optimized_text = solver.optimize_sequence(
            original_text,
            # Hyperparameters for window optimization
            max_window=max_window,
            max_permutations_window = 20000,
            max_attempts_without_improvement_window = 3500,
             # Hyperparameters for global jumps
            max_group_size_jumps = 9, 
            max_attempts_without_improvement_jumps = 3500,
            # Hyperparameters for chunk swaps
            chunk_sizes_swaps = [2, 3, 4, 5, 6, 7, 8, 9], 
            max_attempts_without_improvement_swaps = 3500,
            # Hyperparameters for global shuffling
            n_attempts_shuffle = 20000, 
            max_attempts_without_improvement_shuffle = 3500,
        )
        df.loc[idx, 'text'] = optimized_text

        # Calculate perplexity for the optimized text
        perplexity = scorer.get_perplexity(optimized_text)

        # Add the results to the new DataFrame
        results_df.loc[idx] = [df.loc[idx, 'id'], optimized_text, perplexity]

        print("\n✅ Optimization complete!")
        
    return df, results_df

In [7]:
def set_random_seed(seed: int = None):
    if seed is None:
        seed = random.randint(0, 1000000)
    random.seed(seed)
    print(f"Random seed set to: {seed}")

# Call this function at the beginning of your optimization process
set_random_seed()

Random seed set to: 748921


In [8]:
# Set Pandas display options to show full text
pd.set_option('display.max_colwidth', None)

# Set up paths and constants
MODEL_PATH = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
OUTPUT_PATH = "submission.csv"

print("🎄 Starting Santa 2024 - The Perplexity Permutation Puzzle Solver 🎄")

# Load data
print("\n📚 Loading dataset...")
t = """reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament
reindeer sleep walk the night and drive mistletoe scrooge laugh chimney jump elf bake gingerbread family give advent fireplace ornament
sleigh yuletide beard carol cheer chimney decorations gifts grinch holiday holly jingle magi naughty nice nutcracker ornament polar workshop stocking
sleigh of the magi yuletide cheer is unwrap gifts and eat cheer holiday decorations holly jingle relax sing carol visit workshop grinch naughty nice chimney stocking ornament nutcracker polar beard
from and of to the as in that it we with not you have merry game night season greeting peace angel believe bow card candle candy chocolate cookie doll dream eggnog fireplace fruitcake hohoho hope joy kaggle milk peppermint poinsettia puzzle snowglobe star toy wish wonder workshop wrapping paper wreath
from and and as we and have the in is it of not that the to with you advent card angel bake beard believe bow candy candle carol cheer cheer chocolate chimney cookie decorations doll dream drive eat eggnog family fireplace fireplace chimney fruitcake game give gifts gingerbread greeting grinch holiday holly hohoho hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night night elf nutcracker ornament ornament of the wrapping paper peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wreath yuletide"""


df = pd.DataFrame({'text': t.split('\n')})
df['id'] = df.index # Add the id column based on the index
df = df[['id', 'text']] # Reorder columns to have 'id' first
print(df)

######################################################
# OPTIMIZATION ON THE LATS 3 ROWS !!!
#####################################################
# Slice the DataFrame to include only the last 3 rows
df_last_3 = df.tail(3).reset_index(drop=True)

# Run optimization
try:
    df_last_3_optimized, results_df = optimize_submission(df_last_3, MODEL_PATH)

    # Reconstitute the full dataset by concatenating the non-optimized rows with the optimized rows
    df_non_optimized = df.iloc[:-3]  # All rows except the last 3
    df_reconstituted = pd.concat([df_non_optimized, df_last_3_optimized], ignore_index=True)

    # Save the reconstituted DataFrame to a CSV file
    df_reconstituted.to_csv(OUTPUT_PATH, index=False)

    print("\nResults DataFrame:")
    print(results_df)

    print("\nReconstituted DataFrame:")
    print(df_reconstituted)

    print("\n✅ All done! Good luck in the competition! 🎅")
    
except Exception as e:
    print(f"\n❌ Error occurred: {str(e)}")
    raise

🎄 Starting Santa 2024 - The Perplexity Permutation Puzzle Solver 🎄

📚 Loading dataset...
   id  \
0   0   
1   1   
2   2   
3   3   
4   4   
5   5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           text  
0                                                                                                                                                                                                            

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

✅ Model loaded successfully

🎯 Starting optimization for 3 sequences...


  0%|          | 0/3 [00:00<?, ?it/s]


📝 Processing sequence 1/3
Sequence length: 30 words -> Using max window size: 7

🚀 Starting advanced sequence optimization...
Initial text: sleigh of the magi yuletide cheer is unwrap gifts and eat cheer holiday decorations holly jingle relax sing carol visit workshop grinch naughty nice chimney stocking ornament nutcracker polar beard
Initial perplexity score: 200.62

⏳ Phase: Local Window Optimization
   Time elapsed: 15.55s
   Current best score: 200.62

🔍 Running local window optimization...
   Max window size: 7
   Sequence length: 30 words

   Processing window size 2:
   Window size 2 complete:
      Time taken: 2.89s
      Attempts: 48
      Improvements: 0
      Success rate: 0.00%

   Processing window size 3:
   Window size 3 complete:
      Time taken: 8.52s
      Attempts: 144
      Improvements: 0
      Success rate: 0.00%

   Processing window size 4:
   Window size 4 complete:
      Time taken: 39.90s
      Attempts: 576
      Improvements: 0
      Success rate: 0.00%


...

   Early stopping at window size 7: No improvement in 3500 attempts
   Window size 7 complete:
      Time taken: 173.00s
      Attempts: 3500
      Improvements: 0
      Success rate: 0.00%

📊 Phase Complete: window
   Duration: 457.63s
   Attempts: 21296
   Improvements found: 0

📈 Optimization Summary:
Total time: 1718.57 seconds
Total improvements: 0
Initial score: 200.62
Final score: 200.62
Total improvement: 0.00 (0.00%)

Phase Statistics:
- window:
  • 0 improvements in 457.63s
  • Success rate: 0.00% (0/21296)
- jump:
  • 0 improvements in 310.46s
  • Success rate: 0.00% (0/3504)
- swap:
  • 0 improvements in 20.81s
  • Success rate: 0.00% (0/208)
- shuffle:
  • 0 improvements in 5.93s
  • Success rate: 0.00% (0/50)

✅ Optimization complete!

📝 Processing sequence 2/3
Sequence length: 50 words -> Using max window size: 7

🚀 Starting advanced sequence optimization...
Initial text: from and of to the as in that it we with not you have merry game night season greeting peace angel b

 67%|██████▋   | 2/3 [1:06:25<34:00, 2040.89s/it]

   Early stopping at window size 7: No improvement in 3500 attempts
   Window size 7 complete:
      Time taken: 353.04s
      Attempts: 3500
      Improvements: 0
      Success rate: 0.00%

📊 Phase Complete: window
   Duration: 759.81s
   Attempts: 45112
   Improvements found: 0

📈 Optimization Summary:
Total time: 2266.48 seconds
Total improvements: 0
Initial score: 85.62
Final score: 85.62
Total improvement: 0.00 (0.00%)

Phase Statistics:
- window:
  • 0 improvements in 759.81s
  • Success rate: 0.00% (0/45112)
- jump:
  • 0 improvements in 381.41s
  • Success rate: 0.00% (0/7011)
- swap:
  • 0 improvements in 72.66s
  • Success rate: 0.00% (0/813)
- shuffle:
  • 0 improvements in 6.31s
  • Success rate: 0.00% (0/100)

✅ Optimization complete!

📝 Processing sequence 3/3
Sequence length: 100 words -> Using max window size: 7

🚀 Starting advanced sequence optimization...
Initial text: from and and as we and have the in is it of not that the to with you advent card angel bake beard be

100%|██████████| 3/3 [1:59:41<00:00, 2393.67s/it]

   Early stopping at window size 7: No improvement in 3500 attempts
   Window size 7 complete:
      Time taken: 423.64s
      Attempts: 3500
      Improvements: 0
      Success rate: 0.00%

📊 Phase Complete: window
   Duration: 943.67s
   Attempts: 72128
   Improvements found: 0

📈 Optimization Summary:
Total time: 3195.93 seconds
Total improvements: 0
Initial score: 34.56
Final score: 34.56
Total improvement: 0.00 (0.00%)

Phase Statistics:
- window:
  • 0 improvements in 943.67s
  • Success rate: 0.00% (0/72128)
- jump:
  • 0 improvements in 491.28s
  • Success rate: 0.00% (0/10519)
- swap:
  • 0 improvements in 380.88s
  • Success rate: 0.00% (0/3388)
- shuffle:
  • 0 improvements in 7.57s
  • Success rate: 0.00% (0/150)

✅ Optimization complete!

Results DataFrame:
   id  \
0   3   
1   4   
2   5   

                                                                                                                                                                                      